# Intelligent Hangman Solver - HMM + RL

This notebook implements an intelligent Hangman solver using:
- **Hidden Markov Model (HMM)** for learning letter patterns by position and context
- **Q-Learning (Reinforcement Learning)** agent for decision making

The system trains on a corpus of words and evaluates performance on a test set.

## 1. Import Libraries

In [1]:
import numpy as np
import random
from collections import defaultdict, Counter
import pickle
import json
import os

## 2. Hidden Markov Model Class

The HMM learns letter patterns by position and context, creating separate models for each word length.

In [2]:
class HiddenMarkovModel:
    """
    HMM for Hangman - learns letter patterns by position and context
    """
    def __init__(self):
        self.models_by_length = {}
    
    def train(self, corpus_words):
        """Train HMM on corpus, creating separate models for each word length"""
        print("Training HMM...")
        
        # Group words by length
        words_by_length = defaultdict(list)
        for word in corpus_words:
            word = word.lower().strip()
            if word.isalpha():
                words_by_length[len(word)].append(word)
        
        # Train model for each length
        for length, words in words_by_length.items():
            self.models_by_length[length] = self._train_length_model(words, length)
        
        print(f"Trained HMM for {len(self.models_by_length)} different word lengths")
    
    def _train_length_model(self, words, length):
        """Train model for specific word length"""
        model = {
            'position_freq': defaultdict(Counter),  # position -> letter -> count
            'letter_freq': Counter(),                # letter -> count
            'bigram_freq': Counter(),                # bigram -> count
            'trigram_freq': Counter(),               # trigram -> count
            'total_words': len(words)
        }
        
        for word in words:
            # Position-based frequencies
            for i, char in enumerate(word):
                model['position_freq'][i][char] += 1
                model['letter_freq'][char] += 1
            
            # Bigram frequencies
            for i in range(len(word) - 1):
                bigram = word[i:i+2]
                model['bigram_freq'][bigram] += 1
            
            # Trigram frequencies
            for i in range(len(word) - 2):
                trigram = word[i:i+3]
                model['trigram_freq'][trigram] += 1
        
        return model
    
    def predict(self, masked_word, guessed_letters):
        """
        Predict probability distribution over alphabet given current game state
        Returns: dict of {letter: score}
        """
        length = len(masked_word)
        model = self.models_by_length.get(length)
        
        if not model:
            # Fallback to closest length
            closest_length = min(self.models_by_length.keys(), 
                                key=lambda x: abs(x - length))
            model = self.models_by_length[closest_length]
        
        scores = {}
        alphabet = 'abcdefghijklmnopqrstuvwxyz'
        
        # Count revealed letters for adaptive weighting
        revealed_count = sum(1 for c in masked_word if c != '_')
        progress = revealed_count / length if length > 0 else 0
        
        for char in alphabet:
            if char in guessed_letters:
                continue
            
            score = 0.0
            
            # 1. Position-based scoring (INCREASED weight)
            for i, c in enumerate(masked_word):
                if c == '_':
                    pos_freq = model['position_freq'][i].get(char, 0)
                    score += (pos_freq / model['total_words']) * 3.0  # Increased from 1.0
            
            # 2. Context-based scoring (bigrams) - INCREASED weight
            for i, c in enumerate(masked_word):
                if c == '_':
                    # Left context
                    if i > 0 and masked_word[i-1] != '_':
                        bigram = masked_word[i-1] + char
                        score += model['bigram_freq'].get(bigram, 0) / model['total_words'] * 5.0  # Increased from 2.0
                    
                    # Right context
                    if i < len(masked_word) - 1 and masked_word[i+1] != '_':
                        bigram = char + masked_word[i+1]
                        score += model['bigram_freq'].get(bigram, 0) / model['total_words'] * 5.0  # Increased from 2.0
            
            # 3. Trigram context (INCREASED weight when available)
            for i, c in enumerate(masked_word):
                if c == '_':
                    # _XY pattern
                    if i < len(masked_word) - 2:
                        if masked_word[i+1] != '_' and masked_word[i+2] != '_':
                            trigram = char + masked_word[i+1] + masked_word[i+2]
                            score += model['trigram_freq'].get(trigram, 0) / model['total_words'] * 8.0  # Increased from 3.0
                    
                    # X_Y pattern
                    if i > 0 and i < len(masked_word) - 1:
                        if masked_word[i-1] != '_' and masked_word[i+1] != '_':
                            trigram = masked_word[i-1] + char + masked_word[i+1]
                            score += model['trigram_freq'].get(trigram, 0) / model['total_words'] * 8.0  # Increased from 3.0
                    
                    # XY_ pattern
                    if i > 1:
                        if masked_word[i-2] != '_' and masked_word[i-1] != '_':
                            trigram = masked_word[i-2] + masked_word[i-1] + char
                            score += model['trigram_freq'].get(trigram, 0) / model['total_words'] * 8.0  # Increased from 3.0
            
            # 4. Overall letter frequency (adaptive based on progress)
            # Early game: use more frequency, late game: use less
            freq_weight = 2.0 * (1.0 - progress)  # Decreases as game progresses
            score += model['letter_freq'].get(char, 0) / (model['total_words'] * length) * freq_weight
            
            # 5. Vowel bonus early in the game
            if progress < 0.3 and char in 'aeiou':
                score += 0.5
            
            scores[char] = score
        
        return scores
    
    def save(self, filename):
        """Save model to file"""
        with open(filename, 'wb') as f:
            pickle.dump(self.models_by_length, f)
        print(f"HMM saved to {filename}")
    
    def load(self, filename):
        """Load model from file"""
        with open(filename, 'rb') as f:
            self.models_by_length = pickle.load(f)
        print(f"HMM loaded from {filename}")

## 3. Reinforcement Learning Agent Class

Q-Learning agent that combines learned Q-values with HMM predictions for optimal letter selection.

In [3]:
class RLAgent:
    """
    Q-Learning agent for Hangman
    """
    def __init__(self, hmm, alpha=0.2, gamma=0.98, epsilon=1.0, 
                 epsilon_decay=0.9985, epsilon_min=0.05):
        self.hmm = hmm
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha          # Learning rate (increased from 0.1)
        self.gamma = gamma          # Discount factor (increased from 0.95)
        self.epsilon = epsilon      # Exploration rate
        self.epsilon_decay = epsilon_decay  # Slower decay (was 0.995)
        self.epsilon_min = epsilon_min  # Higher minimum (was 0.01)
        
    def get_state_key(self, masked_word, guessed_letters, lives_left):
        """Create state representation"""
        guessed_str = ''.join(sorted(guessed_letters))
        return f"{masked_word}:{guessed_str}:{lives_left}"
    
    def choose_action(self, masked_word, guessed_letters, lives_left, training=True):
        """
        Choose action using epsilon-greedy with HMM guidance
        """
        hmm_scores = self.hmm.predict(masked_word, guessed_letters)
        available_letters = list(hmm_scores.keys())
        
        if not available_letters:
            return None
        
        state_key = self.get_state_key(masked_word, guessed_letters, lives_left)
        
        # Epsilon-greedy exploration
        if training and random.random() < self.epsilon:
            # Explore: weighted random based on HMM scores
            total_score = sum(hmm_scores.values())
            if total_score > 0:
                probs = [hmm_scores[letter] / total_score for letter in available_letters]
                action = np.random.choice(available_letters, p=probs)
            else:
                action = random.choice(available_letters)
        else:
            # Exploit: combine Q-values with HMM scores
            best_action = None
            best_score = float('-inf')
            
            for letter in available_letters:
                q_value = self.q_table[state_key][letter]
                hmm_score = hmm_scores[letter]
                # Increased HMM weight from 0.3 to 0.7 - HMM is very informative!
                combined_score = q_value + hmm_score * 0.7
                
                if combined_score > best_score:
                    best_score = combined_score
                    best_action = letter
            
            action = best_action
        
        return action
    
    def update_q_value(self, state, action, reward, next_state):
        """Update Q-value using Q-learning formula"""
        current_q = self.q_table[state][action]
        
        # Get max Q-value for next state
        if next_state in self.q_table:
            max_next_q = max(self.q_table[next_state].values()) if self.q_table[next_state] else 0
        else:
            max_next_q = 0
        
        # Q-learning update
        new_q = current_q + self.alpha * (reward + self.gamma * max_next_q - current_q)
        self.q_table[state][action] = new_q
    
    def train(self, corpus_words, episodes=5000, verbose=True):
        """Train agent on corpus - INCREASED episodes"""
        print(f"Training RL agent for {episodes} episodes...")
        
        episode_rewards = []
        episode_wins = []
        
        for episode in range(episodes):
            # Random word from corpus
            word = random.choice(corpus_words).lower().strip()
            if not word.isalpha():
                continue
            
            total_reward = self.play_episode(word, training=True)
            episode_rewards.append(total_reward)
            
            # Decay epsilon
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
            
            if verbose and (episode + 1) % 500 == 0:
                recent_rewards = episode_rewards[-500:]
                avg_reward = np.mean(recent_rewards)
                print(f"Episode {episode + 1}/{episodes} - Avg Reward: {avg_reward:.2f} - Epsilon: {self.epsilon:.3f}")
        
        print("Training complete!")
        return episode_rewards
    
    def play_episode(self, word, training=True):
        """Play one episode of Hangman"""
        masked_word = '_' * len(word)
        guessed_letters = set()
        lives_left = 6
        total_reward = 0
        
        while lives_left > 0 and '_' in masked_word:
            state_key = self.get_state_key(masked_word, guessed_letters, lives_left)
            
            # Choose action
            action = self.choose_action(masked_word, guessed_letters, lives_left, training)
            if action is None:
                break
            
            # Take action
            guessed_letters.add(action)
            
            # Check if correct
            if action in word:
                # Reveal letters
                new_masked = ''.join([c if c in guessed_letters else '_' for c in word])
                revealed_count = new_masked.count('_') - masked_word.count('_')
                revealed_count = abs(revealed_count)
                
                # IMPROVED: Reward scales with letters revealed
                reward = 15 * revealed_count  # Increased from 10
                
                # Bonus for early correct guesses (more lives = better)
                reward += lives_left * 2
                
                # Check if won
                if '_' not in new_masked:
                    reward += 150  # Increased win bonus from 100
                
                masked_word = new_masked
            else:
                # IMPROVED: Penalty increases as lives decrease (avoid risky moves)
                penalty = -25 - (6 - lives_left) * 5  # Gets worse as lives decrease
                reward = penalty
                lives_left -= 1
                
                # Extra penalty if almost lost
                if lives_left == 0:
                    reward -= 100
            
            total_reward += reward
            
            # Update Q-value if training
            if training:
                next_state_key = self.get_state_key(masked_word, guessed_letters, lives_left)
                self.update_q_value(state_key, action, reward, next_state_key)
        
        return total_reward
    
    def play_game(self, word):
        """Play game without training (for evaluation)"""
        word = word.lower().strip()
        masked_word = '_' * len(word)
        guessed_letters = set()
        lives_left = 6
        wrong_guesses = 0
        repeated_guesses = 0
        
        while lives_left > 0 and '_' in masked_word:
            action = self.choose_action(masked_word, guessed_letters, lives_left, training=False)
            if action is None:
                break
            
            # Check for repeated guess
            if action in guessed_letters:
                repeated_guesses += 1
            
            guessed_letters.add(action)
            
            # Check if correct
            if action in word:
                masked_word = ''.join([c if c in guessed_letters else '_' for c in word])
            else:
                wrong_guesses += 1
                lives_left -= 1
        
        won = '_' not in masked_word
        return won, wrong_guesses, repeated_guesses
    
    def save(self, filename):
        """Save Q-table"""
        # Convert defaultdict to regular dict for JSON serialization
        q_table_dict = {k: dict(v) for k, v in self.q_table.items()}
        with open(filename, 'w') as f:
            json.dump(q_table_dict, f)
        print(f"Q-table saved to {filename}")
    
    def load(self, filename):
        """Load Q-table"""
        with open(filename, 'r') as f:
            q_table_dict = json.load(f)
        self.q_table = defaultdict(lambda: defaultdict(float))
        for state, actions in q_table_dict.items():
            for action, value in actions.items():
                self.q_table[state][action] = value
        print(f"Q-table loaded from {filename}")

## 4. Evaluation Function

Function to evaluate the agent's performance on a test set.

In [4]:
def evaluate_agent(agent, test_words, verbose=True):
    """Evaluate agent on test set"""
    print(f"\nEvaluating agent on {len(test_words)} words...")
    
    wins = 0
    total_wrong_guesses = 0
    total_repeated_guesses = 0
    
    for i, word in enumerate(test_words):
        word = word.lower().strip()
        if not word.isalpha():
            continue
        
        won, wrong_guesses, repeated_guesses = agent.play_game(word)
        
        if won:
            wins += 1
        total_wrong_guesses += wrong_guesses
        total_repeated_guesses += repeated_guesses
        
        if verbose and (i + 1) % 500 == 0:
            print(f"Evaluated {i + 1}/{len(test_words)} words...")
    
    total_games = len(test_words)
    success_rate = wins / total_games
    avg_wrong_guesses = total_wrong_guesses / total_games
    
    # Calculate final score using formula: (Success Rate * 2000) - (Total Wrong * 5) - (Total Repeated * 2)
    final_score = (success_rate * 2000) - (total_wrong_guesses * 5) - (total_repeated_guesses * 2)
    
    print("\n" + "="*60)
    print("EVALUATION RESULTS")
    print("="*60)
    print(f"Total Games:           {total_games}")
    print(f"Games Won:             {wins}")
    print(f"Games Lost:            {total_games - wins}")
    print(f"Success Rate:          {success_rate*100:.2f}%")
    print(f"Avg Wrong Guesses:     {avg_wrong_guesses:.2f}")
    print(f"Total Wrong Guesses:   {total_wrong_guesses}")
    print(f"Total Repeated:        {total_repeated_guesses}")
    print(f"\nFINAL SCORE:           {final_score:.2f}")
    print("="*60)
    
    return {
        'total_games': total_games,
        'wins': wins,
        'success_rate': success_rate,
        'avg_wrong_guesses': avg_wrong_guesses,
        'total_wrong_guesses': total_wrong_guesses,
        'total_repeated_guesses': total_repeated_guesses,
        'final_score': final_score
    }

## 5. Load Training Corpus

Load the corpus of words for training the models.

In [5]:
# Load corpus
print("="*60)
print("INTELLIGENT HANGMAN SOLVER - HMM + RL")
print("="*60)

print("\nLoading corpus...")
corpus_path = 'corpus.txt'

try:
    with open(corpus_path, 'r', encoding='utf-8') as f:
        corpus_words = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(corpus_words)} words from corpus")
except FileNotFoundError:
    print(f"Error: Could not find corpus file at {corpus_path}")
    print("Make sure corpus.txt exists in the current directory.")
except Exception as e:
    print(f"Error loading corpus: {str(e)}")

INTELLIGENT HANGMAN SOLVER - HMM + RL

Loading corpus...
Loaded 50000 words from corpus


## 6. Train Hidden Markov Model

Train the HMM on the corpus to learn letter patterns.

In [6]:
# Train HMM
hmm = HiddenMarkovModel()
hmm.train(corpus_words)
hmm.save('hmm_model.pkl')

Training HMM...
Trained HMM for 24 different word lengths
HMM saved to hmm_model.pkl


## 7. Train Reinforcement Learning Agent

Train the Q-Learning agent using the HMM guidance. This may take several minutes.

In [7]:
# Train RL Agent with more episodes for better performance
agent = RLAgent(hmm)
rewards = agent.train(corpus_words, episodes=5000, verbose=True)
agent.save('q_table.json')

Training RL agent for 5000 episodes...
Episode 500/5000 - Avg Reward: -140.27 - Epsilon: 0.473
Episode 1000/5000 - Avg Reward: -83.92 - Epsilon: 0.223
Episode 1500/5000 - Avg Reward: -54.05 - Epsilon: 0.106
Episode 2000/5000 - Avg Reward: -43.07 - Epsilon: 0.050
Episode 2500/5000 - Avg Reward: -45.83 - Epsilon: 0.050
Episode 3000/5000 - Avg Reward: -45.02 - Epsilon: 0.050
Episode 3500/5000 - Avg Reward: -40.11 - Epsilon: 0.050
Episode 4000/5000 - Avg Reward: -42.34 - Epsilon: 0.050
Episode 4500/5000 - Avg Reward: -53.76 - Epsilon: 0.050
Episode 5000/5000 - Avg Reward: -38.72 - Epsilon: 0.050
Training complete!
Q-table saved to q_table.json


## 8. Load Test Set

Load the test words for evaluation.

In [8]:
# Load test set
print("\nLoading test set...")
test_path = 'test.txt'

try:
    with open(test_path, 'r', encoding='utf-8') as f:
        test_words = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(test_words)} test words")
except FileNotFoundError:
    print(f"Error: Could not find test file at {test_path}")
    print("Make sure test.txt exists in the current directory.")
except Exception as e:
    print(f"Error loading test file: {str(e)}")


Loading test set...
Loaded 2000 test words


## 9. Evaluate Agent Performance

Evaluate the trained agent on the test set.

In [9]:
# Evaluate
results = evaluate_agent(agent, test_words)


Evaluating agent on 2000 words...
Evaluated 500/2000 words...
Evaluated 1000/2000 words...
Evaluated 1500/2000 words...
Evaluated 2000/2000 words...

EVALUATION RESULTS
Total Games:           2000
Games Won:             660
Games Lost:            1340
Success Rate:          33.00%
Avg Wrong Guesses:     5.18
Total Wrong Guesses:   10367
Total Repeated:        0

FINAL SCORE:           -51175.00


## 10. Save Results

Save the evaluation results to a JSON file.

In [10]:
# Save results
results_path = 'evaluation_results.json'

try:
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"\nResults saved to {results_path}")
except Exception as e:
    print(f"Error saving results: {str(e)}")


Results saved to evaluation_results.json


## Optional: Test Individual Words

You can test the agent on individual words to see how it plays.

In [11]:
# Example: Test on a specific word
test_word = "python"
won, wrong_guesses, repeated_guesses = agent.play_game(test_word)

print(f"\nTesting word: '{test_word}'")
print(f"Result: {'Won' if won else 'Lost'}")
print(f"Wrong guesses: {wrong_guesses}")
print(f"Repeated guesses: {repeated_guesses}")


Testing word: 'python'
Result: Lost
Wrong guesses: 6
Repeated guesses: 0


## Optional: Load Pre-trained Models

If you have previously saved models, you can load them instead of training from scratch.

In [12]:
# Load pre-trained models (optional)
# hmm_loaded = HiddenMarkovModel()
# hmm_loaded.load('hmm_model.pkl')

# agent_loaded = RLAgent(hmm_loaded)
# agent_loaded.load('q_table.json')

# print("Pre-trained models loaded successfully!")